In [1]:
import pandas as pd

In [2]:
paths = ["focal/ct-mask-finetune-0000"]
for p in paths:
    path = f"/home/dxm060/M3D/LaMed/output/LaMed-Phi3-4B/{p}/eval_caption.csv"
    df = pd.read_csv(path)

    average_values = df[['bleu', 'rouge1', 'meteor']].mean()
    df_avg = pd.DataFrame(average_values).transpose()
    print(df_avg.head())

    top = df.nlargest(100000, 'bleu')[['Ground Truth', 'pred', 'bleu']]
    top.head(100000)

    for index, row in top.iterrows():
        print("Ground Truth")
        print(row['Ground Truth'])  # Print the value from Column1
        print("Prediction")
        print(row['pred'])  # Print the value from Column2
        print()

       bleu    rouge1    meteor
0  0.440885  0.464487  0.495637
Ground Truth
Region: right hilar lymph node
Anatomy: PET/CT axial slice 138
Finding: right hilar lymph node with SUV max of 4.2 (PET/CT axial slice 138), previously it was 4.6.
SUV Max: 4.2
Axial Slice: 138
Prediction
Region: Right hilar lymph node
Anatomy: PET/CT axial slice 138
Finding: Right hilar lymph node with SUV max of 4.0 (PET/CT axial slice 138), previously it was 4.1.
SUV Max: 4.0
Axial Slice: 138

Ground Truth
Region: Right hilar node
Anatomy: axial PET/CT
Finding: Right hilar node with SUV max of 18.9 (axial PET/CT slice 117).
SUV Max: 18.9
Axial Slice: 117
Prediction
Region: Right hilar node
Anatomy: PET/CT axial slice 120
Finding: Right hilar node with SUV max of 17.4 (PET/CT axial slice 120).
SUV Max: 17.4
Axial Slice: 120

Ground Truth
Region: Right inguinal node
Anatomy: Inguinal node
Finding: Right inguinal node measuring 0.9 cm with SUV max of 6.9 (axial PET/CT slice 264).
SUV Max: 6.9
Axial Slice: 264


In [3]:
df.head()

,Question,Ground Truth,pred,bleu,rouge1,meteor
0,<im_patch><im_patch><im_patch><im_patch><im_pa...,Region: Para-aortic lymph node\nAnatomy: N/A\n...,Region: Left periaortic lymph node\nAnatomy: P...,0.600000,0.476190,0.635640
1,<im_patch><im_patch><im_patch><im_patch><im_pa...,Region: N/A\nAnatomy: Left lateral pelvic wall...,Region: Right pelvic sidewall\nAnatomy: Lymph ...,0.396146,0.404762,0.395354
2,<im_patch><im_patch><im_patch><im_patch><im_pa...,Region: L2 left pedicle \nAnatomy: PET/CT axi...,Region: L1 vertebral body\nAnatomy: N/A\nFindi...,0.327586,0.358974,0.466257
3,<im_patch><im_patch><im_patch><im_patch><im_pa...,Region: Left upper paratracheal/paraesophageal...,Region: Left sternum\nAnatomy: Musculoskeletal...,0.283346,0.278481,0.311587
4,<im_patch><im_patch><im_patch><im_patch><im_pa...,Region: Left upper lobe apex\nAnatomy: Chest\n...,Region: Left upper lobe\nAnatomy: Chest\nFindi...,0.647059,0.658537,0.734051


In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Your class map
tsclasses = {
    1: "spleen",
    2: "right kidney",
    3: "left kidney",
    4: "gallbladder",
    5: "liver",
    6: "stomach",
    7: "pancreas",
    8: "right adrenal gland",
    9: "left adrenal gland",
    10: "left upper lung lobe",
    11: "left lower lung lobe",
    12: "right upper lung lobe",
    13: "right middle lung lobe",
    14: "right lower lung lobe",
    15: "esophagus",
    16: "trachea",
    17: "thyroid gland",
    18: "small bowel",
    19: "duodenum",
    20: "colon",
    21: "urinary bladder",
    22: "prostate",
    23: "left kidney cyst",
    24: "right kidney cyst",
    25: "sacrum",
    26: "S1 vertebra",
    27: "L5 vertebra",
    28: "L4 vertebra",
    29: "L3 vertebra",
    30: "L2 vertebra",
    31: "L1 vertebra",
    32: "T12 vertebra",
    33: "T11 vertebra",
    34: "T10 vertebra",
    35: "T9 vertebra",
    36: "T8 vertebra",
    37: "T7 vertebra",
    38: "T6 vertebra",
    39: "T5 vertebra",
    40: "T4 vertebra",
    41: "T3 vertebra",
    42: "T2 vertebra",
    43: "T1 vertebra",
    44: "C7 vertebra",
    45: "C6 vertebra",
    46: "C5 vertebra",
    47: "C4 vertebra",
    48: "C3 vertebra",
    49: "C2 vertebra",
    50: "C1 vertebra",
    51: "heart",
    52: "aorta",
    53: "pulmonary vein",
    54: "brachiocephalic trunk",
    55: "right subclavian artery",
    56: "left subclavian artery",
    57: "right common carotid artery",
    58: "left common carotid artery",
    59: "left brachiocephalic vein",
    60: "right brachiocephalic vein",
    61: "left atrial appendage",
    62: "superior vena cava",
    63: "inferior vena cava",
    64: "portal and splenic veins",
    65: "left iliac artery",
    66: "right iliac artery",
    67: "left iliac vein",
    68: "right iliac vein",
    69: "left humerus",
    70: "right humerus",
    71: "left scapula",
    72: "right scapula",
    73: "left clavicle",
    74: "right clavicle",
    75: "left femur",
    76: "right femur",
    77: "left hip",
    78: "right hip",
    79: "spinal cord",
    80: "left gluteus maximus",
    81: "right gluteus maximus",
    82: "left gluteus medius",
    83: "right gluteus medius",
    84: "left gluteus minimus",
    85: "right gluteus minimus",
    86: "left autochthonous back muscle",
    87: "right autochthonous back muscle",
    88: "left iliopsoas",
    89: "right iliopsoas",
    90: "brain",
    91: "skull",
    92: "left 1st rib",
    93: "left 2nd rib",
    94: "left 3rd rib",
    95: "left 4th rib",
    96: "left 5th rib",
    97: "left 6th rib",
    98: "left 7th rib",
    99: "left 8th rib",
    100: "left 9th rib",
    101: "left 10th rib",
    102: "left 11th rib",
    103: "left 12th rib",
    104: "right 1st rib",
    105: "right 2nd rib",
    106: "right 3rd rib",
    107: "right 4th rib",
    108: "right 5th rib",
    109: "right 6th rib",
    110: "right 7th rib",
    111: "right 8th rib",
    112: "right 9th rib",
    113: "right 10th rib",
    114: "right 11th rib",
    115: "right 12th rib",
    116: "sternum",
    117: "costal cartilages"
}
# Invert the class map: class_name -> ID
name_to_id = {v.lower(): k for k, v in tsclasses.items()}

# Function to extract the class label from a sentence
def extract_class(sentence):
    sentence = sentence.lower()
    for name in name_to_id:
        if name in sentence:
            return name
    return None

# Extract predicted and true labels
df['gt_class'] = df['Ground Truth'].apply(extract_class)
df['pred_class'] = df['pred'].apply(extract_class)

# Create binary lists for evaluation (assuming 1-to-1 match)
df['correct'] = df['gt_class'] == df['pred_class']

# Drop rows where class couldn't be matched (optional)
filtered_df = df#.dropna(subset=['gt_class', 'pred_class'])

# Binary labels
y_true = filtered_df['gt_class']
y_pred = filtered_df['pred_class']

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')

print(f"Accuracy:  {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")


Accuracy:  0.912
Precision: 0.916
Recall:    0.912


In [14]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score

# Extract predicted and true labels
df['gt_class'] = df['Ground Truth'].apply(extract_class)
df['pred_class'] = df['pred'].apply(extract_class)

# Drop rows where class couldn't be matched
filtered_df = df.dropna(subset=['gt_class', 'pred_class'])

# Ground truth and predictions
y_true = filtered_df['gt_class']
y_pred = filtered_df['pred_class']

# Compute overall metrics
overall_accuracy = accuracy_score(y_true, y_pred)
overall_precision = precision_score(y_true, y_pred, average='macro')
overall_recall = recall_score(y_true, y_pred, average='macro')

print(f"\nOverall Accuracy:  {overall_accuracy:.3f}")
print(f"Overall Precision: {overall_precision:.3f}")
print(f"Overall Recall:    {overall_recall:.3f}\n")

# Classification report
report = classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

# Filter to actual classes
class_names = sorted(set(y_true.unique()) | set(y_pred.unique()))
class_metrics_df = report_df.loc[class_names, ['precision', 'recall', 'f1-score', 'support']].round(3)

# Compute per-class accuracy manually
per_class_accuracy = {}
for cls in class_names:
    cls_mask = (y_true == cls)
    correct_preds = (y_true[cls_mask] == y_pred[cls_mask]).sum()
    total_preds = cls_mask.sum()
    acc = correct_preds / total_preds if total_preds > 0 else 0
    per_class_accuracy[cls] = round(acc, 3)

# Add accuracy column
class_metrics_df['accuracy'] = class_metrics_df.index.map(per_class_accuracy)

# Print per-class metrics
# Sort the DataFrame by accuracy (descending)
class_metrics_df = class_metrics_df.sort_values(by='accuracy', ascending=False)

# Print sorted per-class metrics
print("Per-Class Metrics (sorted by accuracy):\n")
print(class_metrics_df)
class_metrics_df.to_csv("per_class_metrics.csv", index=True)



Overall Accuracy:  0.912
Overall Precision: 0.916
Overall Recall:    0.912

Per-Class Metrics (sorted by accuracy):

                   precision  recall  f1-score  support  accuracy
trachea                0.980    1.00     0.990     50.0      1.00
aorta                  1.000    1.00     1.000     50.0      1.00
stomach                1.000    1.00     1.000     50.0      1.00
costal cartilages      1.000    1.00     1.000     50.0      1.00
esophagus              1.000    1.00     1.000     50.0      1.00
...                      ...     ...       ...      ...       ...
t10 vertebra           0.542    0.64     0.587     50.0      0.64
right 8th rib          0.640    0.64     0.640     50.0      0.64
right iliac vein       0.705    0.62     0.660     50.0      0.62
t9 vertebra            0.757    0.56     0.644     50.0      0.56
t11 vertebra           0.634    0.52     0.571     50.0      0.52

[115 rows x 5 columns]
